# H02 Tensorflow Lite Helpers

In [1]:
def convert_tf_model(tf_model, tf_model_file, quantization_type, representative_dataset, input_dt='float32', output_dt='float32', pure_int=False):

    def yield_representative_dataset():
      for x in representative_dataset:
        yield [np.array([x], dtype=np.float32)]
    
    converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)

    if input_dt == 'int8':
        converter.inference_input_type = tf.int8  # or tf.uint8
    if output_dt == 'int8':
        converter.inference_output_type = tf.int8  # or tf.uint8
        
    # foat32.
    if quantization_type == 'none':
        tflite_model = converter.convert()        
        filename = './TFLite-model/' + get_tf_model_string(tf_model_file) + '_Q-none' + '.tflite'
        open(filename, 'wb').write(tflite_model)
        print(f'The model was sucessfully converted and written to {filename}.')
        return filename
    
    elif quantization_type == 'mixed':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        filename = './TFLite-model/' + get_tf_model_string(tf_model_file) + '_Q-mixed' + '.tflite'
        open(filename, 'wb').write(tflite_model)
        print(f'The model was sucessfully converted, quantized and written to {filename}.')
        return filename
        
        
    elif quantization_type == 'full':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        
        if pure_int:
            # Note: The converter will throw an error
            # if it encounters an operation it cannot currently quantize.
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
            filename = './TFLite-model/' + get_tf_model_string(tf_model_file) + '_Q-full_IO-int8' + '.tflite'
        else:
            filename = './TFLite-model/' + get_tf_model_string(tf_model_file) + '_Q-full' + '.tflite'


        
        converter.representative_dataset = yield_representative_dataset
        tflite_model = converter.convert()
        open(filename, 'wb').write(tflite_model)
        print(f'The model was sucessfully converted, quantized and written to {filename}.')
        return filename

    else:
        print("Unkown quantization.")

In [10]:
def get_tfl_size(tfl_model_file, unquantized_model_file=None, gzip=False):
    
    if unquantized_model_file:
        basic_model_size = os.path.getsize(unquantized_model_file) / 1024
    else:
        basic_model_size = os.path.getsize(tfl_model_file) / 1024
    
    
    if gzip:
        import zipfile
        zipped_file = './keras-model/temp_zipped.zip'
        with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
            f.write(tfl_model_file)
        model_size = os.path.getsize(zipped_file) / 1024
    else:
        model_size = os.path.getsize(tfl_model_file) / 1024
        
    reduction = 100*((basic_model_size - model_size) / basic_model_size)
    return model_size, reduction

In [62]:
def tfl_predict(tfl_model_file, x):
    
    model = tf.lite.Interpreter(tfl_model_file)
    model.allocate_tensors()
    
    model_input = model.tensor(model.get_input_details()[0]["index"])
    model_output = model.tensor(model.get_output_details()[0]["index"])
    
    input_details = model.get_input_details()
    output_details = model.get_output_details()

    # this might not be competly dynamic
    predictions = np.empty((x.shape[0], output_details[0]['shape'][1]), dtype=np.float32)
    
    
    for index, data in enumerate(x):
        input_data = np.array([data], dtype=np.float32)
        model.set_tensor(input_details[0]['index'], input_data)
        model.invoke()
        output_data = model.get_tensor(output_details[0]['index'])
        predictions[index] = output_data
        
    return predictions

In [41]:
def calc_accuracy(tfl_model_predictions, y):
    accurate_count = 0

    for index in range(len(tfl_model_predictions)):
        if np.argmax(tfl_model_predictions[index]) == np.argmax(y[index]):
            accurate_count += 1

    model_accuracy = accurate_count * 1.0 / len(tfl_model_predictions)
    
    return model_accuracy

In [57]:
def get_tfl_details(tfl_model_file):
    
    model = tf.lite.Interpreter(tfl_model_file)
    model.allocate_tensors()
    
    input_details = model.get_input_details()
    output_details = model.get_output_details()

    # input_shape = input_details[0]['shape']
    # print("\ninput_shape: ", input_shape)

    # output_shape = output_details[0]['shape']
    # print("\noutput_shape: ", output_shape)

    return input_details, output_details

In [25]:
def get_tfl_layer_details(tfl_model_file):
    model = tf.lite.Interpreter(tfl_model_file)
    model.allocate_tensors()
    
    tensor_details = model.get_tensor_details()

    return tensor_details

In [49]:
def tfl_inference(tfl_model_file, input_data):

    input_data = np.array([input_data], dtype=np.float32)

    model = tf.lite.Interpreter(tfl_model_file)
    model.allocate_tensors()
    input_details, output_details = get_tfl_details(tfl_model_file)
    model.set_tensor(input_details[0]['index'], input_data)

    model.invoke()
    model_prediction = model.get_tensor(output_details[0]['index'])
    model.reset_all_variables()

    # show image
    plt.imshow(x_test_normalized[image_no].squeeze(), cmap=plt.cm.gray_r)
    plt.show()
    
    # print prediction
    print(model_prediction)
    plt.figure()
    plt.bar(np.arange(10), model_prediction[0])
    plt.show()

In [ ]:
print("Imported helper functions from H02_TFL-Conversion.ipynb")